In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import glob

In [14]:
# Concatenate
df = []
for xl in glob.glob("data_geo/Metadata*.xlsx"):
    print(xl)
    idf = pd.read_excel(xl)
    idf["hp_petugas"] = idf["hp_petugas"].astype("str").replace({"nan":np.nan})
    idf["type"] = xl.split("\\")[1].split(".")[0].split(" ")[1]
    idf["sumber"] = xl.split("\\")[1]
    df.append(idf)

df = pd.concat(df, axis=0, join="outer", ignore_index=True)

# Corrections
df.loc[ (df["elv_station"] == "-"), "elv_station"] = np.nan
df.loc[ (df["elv_station"] == "15\\r\\n\\r\\n"), "elv_station"] = 15
df.loc[ (df["nama_propinsi"] == "Nanggroe Aceh Darusalam"), "nama_propinsi"] = "Aceh"

for col in [f"{x}_station" for x in ["long","latt","elv"]]:
    print(col)
    df[col] = df[col].astype(np.float32)

# Add geometry column
df = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(
        x= df["long_station"], 
        y= df["latt_station"], 
        # z= df["elv_station"], 
        crs="EPSG:4326"
        )
    )
print(df.columns)
df = df.sort_values(["type", "nama_propinsi", "nama_kota", "name_station"]).set_index(["type", "name_station"])

# Null elevation
df_null_elv = df["elv_station"].isna()

# Export
df.to_file("data_geo/Metadata.geojson")
df.to_file("data_geo/Metadata.gpkg")

# for col in df.index.get_level_values(0).unique():
#     df.loc[col].to_file(f"data_geo/Metadata_{col}.geojson")

df

data_geo\Metadata AAWS.xlsx
data_geo\Metadata ARG.xlsx
data_geo\Metadata ASRS.xlsx
data_geo\Metadata AWS.xlsx
data_geo\Metadata IKRO.xlsx
long_station
latt_station
elv_station
Index(['id_station', 'name_station', 'nama_propinsi', 'nama_kota', 'kecamatan',
       'kelurahan', 'latt_station', 'long_station', 'elv_station',
       'status_operasional', 'hp_petugas', 'tgl_pasang', 'addr_instansi',
       'data_transport', 'instansi', 'nama_vendor', 'type', 'sumber',
       'geometry'],
      dtype='object')


id_station      nama_propinsi  \
type name_station                                                              
AAWS AAWS Kutacane                                STA3208               Aceh   
     AAWS Peureulak                               STA3031               Aceh   
     AAWS Peudada                                 STA3227               Aceh   
     AAWS Nagan Raya                              STA3225               Aceh   
     AAWS Karangasem                              STA3055               Bali   
...                                                   ...                ...   
IKRO IKRO Stasiun Klimatologi Sulawesi Selatan      50020   Sulawesi Selatan   
     Ikro Stasiun Klimatologi Ranomeeto             50012  Sulawesi Tenggara   
     IKRO Staklim Sulawesi Utara                  STI1006     Sulawesi Utara   
     Stasiun Klimatologi Padang Priaman             50002     Sumatera Barat   
     Stasiun Klimatologi Deli Serdang               50019     Sumatera Utara   

                                                           nama_kota  \
type name_station                                                      
AAWS AAWS Kutacane                                Kab. Aceh Tenggara   
     AAWS Peureulak                                  Kab. Aceh Timur   
     AAWS Peudada                                        Kab. Bireun   
     AAWS Nagan Raya                                 Kab. Nagan Raya   
     AAWS Karangasem                                Kab. Karang Asem   
...                                                              ...   
IKRO IKRO Stasiun Klimatologi Sulawesi Selatan            Kab. Maros   
     Ikro Stasiun Klimatologi Ranomeeto          Kab. Konawe Selatan   
     IKRO Staklim Sulawesi Utara                 Kab. Minahasa Utara   
     Stasiun Klimatologi Padang Priaman         Kab. Padang Pariaman   
     Stasiun Klimatologi Deli Serdang               Kab. Deliserdang   

                                                      kecamatan  \
type name_station                                                 
AAWS AAWS Kutacane                                        Badar   
     AAWS Peureulak                                   Peureulak   
     AAWS Peudada                                       Peudada   
     AAWS Nagan Raya                                      Kuala   
     AAWS Karangasem                                    Rendang   
...                                                         ...   
IKRO IKRO Stasiun Klimatologi Sulawesi Selatan       Maros Baru   
     Ikro Stasiun Klimatologi Ranomeeto               Ranomeeto   
     IKRO Staklim Sulawesi Utara                       Talawaan   
     Stasiun Klimatologi Padang Priaman              Kayu Tanam   
     Stasiun Klimatologi Deli Serdang           Percut Sei tuan   

                                                      kelurahan  latt_station  \
type name_station                                                               
AAWS AAWS Kutacane                                          NaN      3.524524   
     AAWS Peureulak                                Beusa Merano      4.827319   
     AAWS Peudada                                           NaN      5.195601   
     AAWS Nagan Raya                                        NaN      4.086564   
     AAWS Karangasem                                  Singarata     -8.415980   
...                                                         ...           ...   
IKRO IKRO Stasiun Klimatologi Sulawesi Selatan       BAJU BODOA     -4.997013   
     Ikro Stasiun Klimatologi Ranomeeto                 Ranooha     -4.055833   
     IKRO Staklim Sulawesi Utara                    Paniki Atas      1.500839   
     Stasiun Klimatologi Padang Priaman         Kapalo Hilalang     -0.545850   
     Stasiun Klimatologi Deli Serdang                   Sampali      3.621352   

                                                long_station  elv_station  \
type name_station                                                 

In [15]:
gdf_test = gpd.read_file("data_geo/Metadata.gpkg")
gdf_test["nama_propinsi"].unique()

gdf_test = gdf_test[~((gdf_test["type"] == "AWS") & (gdf_test["nama_propinsi"].str.contains("Kalimantan")))]
gdf_test = gdf_test[~((gdf_test["type"] == "IKRO") & (~gdf_test["nama_propinsi"].str.contains("Sulawesi")))]
gdf_test.to_file("data_geo/Metadata_tes_provIntersect.geojson")
gdf_test.to_file("data_geo/Metadata_tes_provIntersect.gpkg")
gdf_test

,type,name_station,id_station,nama_propinsi,nama_kota,kecamatan,kelurahan,latt_station,long_station,elv_station,status_operasional,hp_petugas,tgl_pasang,addr_instansi,data_transport,instansi,nama_vendor,sumber,geometry
0,AAWS,AAWS Kutacane,STA3208,Aceh,Kab. Aceh Tenggara,Badar,None,3.524524,97.784897,221.0,operasional,82362282937,2016-09-03,Dinas Pertanian Aceh Tenggara,FTP,Dinas Pertanian Aceh Tenggara,Theis,Metadata AAWS.xlsx,POINT (97.7849 3.52452)
1,AAWS,AAWS Peureulak,STA3031,Aceh,Kab. Aceh Timur,Peureulak,Beusa Merano,4.827319,97.890518,6.0,operasional,85276200598,2016-09-13,Jl.Banda Aceh-Medan Km. 393,FTP,Laboratorium PHP Peureulak,BMKG-MSP,Metadata AAWS.xlsx,POINT (97.89052 4.82732)
2,AAWS,AAWS Peudada,STA3227,Aceh,Kab. Bireun,Peudada,None,5.195601,96.594353,8.0,operasional,81263332400,2016-09-10,BP3K Peudada,FTP,BP3K Peudada,Theis,Metadata AAWS.xlsx,POINT (96.59435 5.1956)
3,AAWS,AAWS Nagan Raya,STA3225,Aceh,Kab. Nagan Raya,Kuala,None,4.086564,96.275452,11.0,operasional,85260473599,2016-09-10,BPP Pulo Ie,FTP,BPP Pulo Ie,Theis,Metadata AAWS.xlsx,POINT (96.27545 4.08656)
4,AAWS,AAWS Karangasem,STA3055,Bali,Kab. Karang Asem,Rendang,Singarata,-8.415980,115.420097,590.0,operasional,85792135188,2016-09-10,None,FTP,UPTD BP3 Kab. Karangasem,Theis,Metadata AAWS.xlsx,POINT (115.4201 -8.41598)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,AWS,AWS Parapat,160044,Sumatera Utara,Kab. Simalungun,Girsang SSimpang Bolon,Sibaganding,2.662900,98.939598,970.0,operasional,081370859314,2017-11-14,Jl. Lintas Sumatera - Parapat,FTP,Komplek Rumah Dinas Stasiun Geofisika Parapat,MMI-LSI,Metadata AWS.xlsx,POINT (98.9396 2.6629)
1264,AWS,AWS Digi Stamet Kualanamu Medan,STA5001,Sumatera Utara,Kota Medan,Tanjung Morawa,Penara Kebun,3.645760,98.884903,23.0,operasional,None,2016-09-10,Bandara Kualanamu,FTP,BMKG-Kualanamu,Arkan-Digitalisasi,Metadata AWS.xlsx,POINT (98.8849 3.64576)
1286,IKRO,IKRO Stasiun Klimatologi Sulawesi Selatan,50020,Sulawesi Selatan,Kab. Maros,Maros Baru,BAJU BODOA,-4.997013,119.571739,13.0,operasional,None,2020-10-16,JALAN DR RATULANGI NO 75 A MAROS,None,STASIUN KLIMATOLOGI KELAS I MAROS,None,Metadata IKRO.xlsx,POINT (119.57174 -4.99701)
1287,IKRO,Ikro Stasiun Klimatologi Ranomeeto,50012,Sulawesi Tenggara,Kab. Konawe Selatan,Ranomeeto,Ranooha,-4.055833,122.449165,NaN,operasional,081343897908,2018-12-07,Jl. Poros Bandara Haluoleo,None,Stasiun Klimatologi Ranomeeto,None,Metadata IKRO.xlsx,POINT (122.44917 -4.05583)


In [16]:
import folium
import geopandas as gpd

In [18]:
gdf = gpd.read_file("data_geo/Metadata.geojson")
# gdf.set_index(["type","name_station"])#.tail(3)
gdf[gdf["geometry"] == None]

,type,name_station,id_station,nama_propinsi,nama_kota,kecamatan,kelurahan,latt_station,long_station,elv_station,status_operasional,hp_petugas,tgl_pasang,addr_instansi,data_transport,instansi,nama_vendor,sumber,geometry


In [23]:
ALAT_DESCS = {
    "AWS": {
        "full name":"Automatic Weather Station",
        "icon":":cloud:", 
        "icon_fa":"cloud",
        "color":"blue",
    },
    "AAWS": {
        "full name":"Automatic Agroclimate Weather Station",
        "icon":":corn:",
        "icon_fa":"leaf",
        "color":"green",
    },
    "ARG": {
        "full name":"Automatic Rain Gauge",
        "icon":":droplet:",
        "icon_fa":"droplet",
        "color":"darkblue",
    },
    "IKRO": {
        "full name":"Pengamatan Iklim Mikro",
        "icon":":golf:",
        "icon_fa":"map-signs",
        "color":"purple",
    },
    "ASRS": {
        "full name":"Automatic Solar Radiation Station",
        "icon":":high_brightness:", 
        "icon_fa":"sun",
        "color":"orange",
    },
}

m = folium.Map(
    location=(-2,117), 
    tiles="cartodb positron", 
    zoom_start=5,
    control_scale=True,
    width="100%",
    height="75%",
    min_lat=-12, max_lat=8,
    min_lon=93, max_lon=143,
    max_bounds=True,
)

for ftype in ALAT_DESCS.keys():
    fgdf = gdf[gdf["type"] == ftype]
    print(ftype, len(fgdf))

    if len(fgdf)>0:
        fg = folium.FeatureGroup(name=ftype, overlay=True, show=True).add_to(m)

        ficon = folium.Icon(
            color=ALAT_DESCS[ftype]["color"],
            icon=ALAT_DESCS[ftype]["icon_fa"], 
            prefix="fa",
        )

        for pts in fgdf.itertuples():
            folium.Marker(
                location=(float(pts.latt_station), float(pts.long_station)),
                popup=f"{pts.id_station} {pts.name_station} {pts.type}",
                icon=ficon,
            ).add_to(fg)

folium.LayerControl(collapsed=False).add_to(m)

m

AWS 384
AAWS 104
ARG 746
IKRO 26
ASRS 31


In [53]:
opts = gdf["type"].unique().tolist() + ["xxx"]
diff = set(opts)-set(ALAT_DESCS.keys())
join = list(ALAT_DESCS.keys()) + list(diff)
join

['AWS', 'AAWS', 'ARG', 'IKRO', 'ASRS', 'xxx']

In [48]:
opts

array(['ARG', 'ASRS', 'AWS', 'IKRO'], dtype=object)